In [34]:
from centerline import CenterLine, smooth_mask, iou
from scipy import io as sio
from glob import glob
from tqdm.notebook import tqdm
from skimage import io, morphology, img_as_float, filters
import numpy as np
import math

In [2]:
### Process a batch of network output
raw_nw_fnames = glob('evaluations/pilot/network_output/resTe/*_Lpred.png')
raw_input_fnames = glob('evaluations/pilot/network_output/resTe/*_I.png')
raw_target_fnames = glob('evaluations/pilot/network_output/resTe/*_Ltrue.png')
mat_fnames = glob('evaluations/pilot/ctFIRE_dir/ctFIREout/*.mat')

ridge_feats = []
mat_feats = []
nw_feats = []
truth_feats = []

for i in tqdm(range(len(raw_input_fnames))):
    nw_fname = raw_nw_fnames[i]
    input_fname = raw_input_fnames[i]
    target_fname = raw_target_fnames[i]
    mat_fname = mat_fnames[i]
    assc_img = io.imread(input_fname)

    # ground truth
    centerline = CenterLine(centerline_image=io.imread(target_fname), associate_image=assc_img)
    centerline.compute_fiber_feats() 
    truth_feats.append(centerline.feats)

    # vectorization using ridge detector
    img = img_as_float( io.imread( nw_fname ))
    binary = img > filters.threshold_mean((img))
    centerline_res = CenterLine(associate_image=binary)
    line_dict = centerline_res.ridge_detector(config_fname="ridge_detector_params.json")
    centerline_res = CenterLine(line_dict=line_dict, associate_image=assc_img)
    centerline_res.compute_fiber_feats() 
    nw_feats.append(centerline_res.feats)

    # ridge detector
    centerline_ridge = CenterLine(associate_image=assc_img)
    line_dict = centerline_ridge.ridge_detector(config_fname="ridge_detector_params.json")
    centerline_ridge = CenterLine(line_dict=line_dict, associate_image=assc_img)
    centerline_ridge.compute_fiber_feats() 
    ridge_feats.append(centerline_ridge.feats)

    # ctFIRE
    centerline_mat = CenterLine()
    mat = sio.loadmat(mat_fname)
    mat_data = mat['data']
    line_dict = centerline_mat.mat_to_lines(mat_data=mat_data)
    centerline_mat = CenterLine(line_dict=line_dict, image_size=(256, 256))
    centerline_mat.compute_fiber_feats() 
    mat_feats.append(centerline_mat.feats)

  0%|          | 0/201 [00:00<?, ?it/s]

At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dataframe should be provided.
At least one of centerline_image, line_dict, and dat

In [89]:
def feats_diff(feats_in, feats_ref, abs_val=True):
    diff = []
    cir_dist = lambda x, y, r=math.pi: min(r - abs(x - y), abs(x - y))
    for k, v in feats_in.items():
        if k == 'cir_mean':
            diff.append(cir_dist(feats_in[k], feats_ref[k]))
        elif k == 'density':
            ratio, U, I = iou(feats_in[k], feats_ref[k])
            diff.append(ratio)
        else:
            if abs_val: 
                diff.append(abs(feats_in[k] - feats_ref[k]))
            else: 
                diff.append(feats_in[k] - feats_ref[k])
    return diff


In [90]:
ridge_diffs = []
mat_diffs = []
nw_diffs = []
for i in range(len(ridge_feats)):
    ridge_diffs.append(feats_diff(ridge_feats[i], truth_feats[i]))
    mat_diffs.append(feats_diff(mat_feats[i], truth_feats[i]))
    nw_diffs.append(feats_diff(nw_feats[i], truth_feats[i]))
ridge_diffs = np.stack(ridge_diffs)
mat_diffs = np.stack(mat_diffs)
nw_diffs = np.stack(nw_diffs)

In [91]:
cir_var = 0
len_mean = 0
len_var = 0
waviness = 0
intensity = 0
for i in truth_feats:
    cir_var = max(cir_var, i['cir_var'])
    len_mean = max(len_mean, i['len_mean'])
    len_var = max(len_var, i['len_var'])
    waviness = max(waviness, i['waviness'])
    intensity = max(intensity, i['intensity'])
max_diff = [math.pi, cir_var, len_mean, len_var, waviness, intensity]
max_diff

[3.141592653589793,
 1.4878305075685978,
 135.875,
 80.52396770527393,
 0.394363115949018,
 4711]

In [92]:
normalize_error = lambda diff, max_diff: [diff[i]/max_diff[i] if i<6 else diff[i] for i in range(7)]
norm_ridge_diffs = np.array([normalize_error(i, max_diff) for i in ridge_diffs])
norm_mat_diffs = np.array([normalize_error(i, max_diff) for i in mat_diffs])
norm_nw_diffs = np.array([normalize_error(i, max_diff) for i in nw_diffs])

In [98]:
summary = lambda name, diffs: print(f'{name} normalized error : cir_mean {diffs[0]:.4f} cir_var {diffs[1]:.4f} len_mean {diffs[2]:.4f} len_var {diffs[3]:.4f} waviness {diffs[4]:.4f} intensity {diffs[5]:.4f} || IoU: {diffs[6]:.4f}')
summary('Ridge', np.mean(norm_ridge_diffs, 0))
summary('ctFIRE',np.mean(norm_mat_diffs, 0))
summary('Network',np.mean(norm_nw_diffs, 0))

Ridge normalized error  : cir_mean 0.0555 cir_var 0.1710 len_mean 0.0752 len_var 0.0757 waviness 0.2027 intensity 0.1982 || IoU: 0.6514
ctFIRE normalized error  : cir_mean 0.0579 cir_var 0.1362 len_mean 0.0596 len_var 0.0871 waviness 0.1426 intensity 0.1279 || IoU: 0.6507
Network normalized error  : cir_mean 0.0571 cir_var 0.1115 len_mean 0.0431 len_var 0.0581 waviness 0.1430 intensity 0.1077 || IoU: 0.6967


In [3]:
import pickle
with open('evaluations/pilot/ridge_feats.pkl', 'wb') as f:
    pickle.dump(ridge_feats, f)
        
with open('evaluations/pilot/mat_feats.pkl', 'wb') as f:
    pickle.dump(mat_feats, f)

with open('evaluations/pilot/nw_feats.pkl', 'wb') as f:
    pickle.dump(nw_feats, f)

with open('evaluations/pilot/truth_feats.pkl', 'wb') as f:
    pickle.dump(truth_feats, f)

In [105]:
diffs_pack = np.dstack([norm_ridge_diffs, norm_mat_diffs, norm_nw_diffs])

In [118]:
pack = diffs_pack[:, :6, :]
sample_scores = []
for i in range(pack.shape[0]):
    stack = pack[i, :, :]
    ranks =  np.argsort(stack, 1)
    scores = np.mean(ranks, 0)
    sample_scores.append(scores)
sample_scores = np.vstack(sample_scores)

In [122]:
rank_score = np.mean(sample_scores, 0)
print(f'Average rank score: Ridge {rank_score[0]:.4f} ctFIRE {rank_score[1]:.4f} Network {rank_score[2]:.4f}')

Average rank score: Ridge 1.2521 ctFIRE 1.0887 Network 0.6592
